# Block 6 Exercise 2: finding the best parameters for predicting the fare of taxi rides
We return to our Random Forest Regression and want to automatically optimize all free parameters ...

In [ ]:
import pandas as pd
import numpy as np
import folium


In [ ]:
#check if notebook runs in colab
import sys
IN_COLAB = 'google.colab' in sys.modules
print('running in Colab:',IN_COLAB)
path='..'
if IN_COLAB:
  #in colab, we need to clone the data from the repo
  !git clone https://github.com/keuperj/DataScienceSS20.git
  path='DataScienceSS20'

In [ ]:
# we load the data we have saved after wrangling and pre-processing in block I
X=pd.read_csv(path+'/DATA/train_cleaned.csv')
drop_columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','key','pickup_datetime','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3']
X=X.drop(drop_columns,axis=1)
X=pd.get_dummies(X)# one hot coding
#generate labels
y=X['fare_amount']
X=X.drop(['fare_amount'],axis=1)

### Scikit Optimize
Scikit Optimize (https://scikit-optimize.github.io/stable/index.html) is a AutoML toolbox wrapped around Scikit-Learn. It allows us to use state-of-the-art automatic hyper-parameter optimization on top of our learning algorithms.   



In [ ]:
# install 
!pip install scikit-optimize

### E 2.1 Bayesian Optimization of a Random Forest Regression Model
use Bayesian Optimization with Cross-Validation (https://scikit-optimize.github.io/stable/modules/generated/skopt.BayesSearchCV.html#skopt.BayesSearchCV) to find the best regression model. Compare
* linear regression (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) 
* Random Forest regression (https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor)
* and SVM regression (https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR)

NOTES: this can become quite compute intensive! Hence,
* use a smaller subset of the training data to run the experiments 
* think about the range of your parameters (e.g. larger number of trees in RF or high C-values in SMV will make models expensive)
* optimize only the following parameters per model type:
    * linear: no parameters to optimize
    * RF: #trees and depth
    * SVM: C and gamma (use RBF kernel)
* parallelize -> n_jobs
* use CoLab to rum the job for up to 12h 


In [ ]:
from skopt import BayesSearchCV

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.to_numpy()[:1000,:], y.to_numpy()[:1000])

In [ ]:
#RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
rfr = BayesSearchCV(
    RandomForestRegressor(),  #estimator
    #search_spaces
    {
        'max_depth':(2,20),
        'n_estimators':(10,200)
    },
    n_iter=32, #Number of parameter settings that are sampled.
    random_state=0,
    cv=5, #Determines the cross-validation splitting strategy.
    n_jobs=4 #number of jobs to run parallel
)

In [ ]:
rfr.fit(X_train, y_train)

In [ ]:
rfr.best_score_

In [ ]:
rfr.best_params_

In [ ]:
#SVM
from sklearn.svm import SVR
from skopt.space import Real, Categorical, Integer
svm = BayesSearchCV(
    SVR(),
    {
        'C':Real(1e-6, 1e+6, prior='log-uniform'),
        'gamma':Real(1e-6, 1e+1, prior='log-uniform')  
    },
    n_iter=32, #Number of parameter settings that are sampled.
    random_state=0,
    cv=5, #Determines the cross-validation splitting strategy.
    n_jobs=4 #number of jobs to run parallel
)
    

In [ ]:
svm.fit(X_train, y_train)

In [ ]:
svm.best_score_

In [ ]:
svm.best_params_

In [ ]:
#LinearRegression
#from sklearn.linear_model import LinearRegression
#from skopt.space import Real, Categorical, Integer
#linear = BayesSearchCV(
  #  LinearRegression(),
  #  {?},
  #  n_iter=32, #Number of parameter settings that are sampled.
  #  random_state=0,
  #  cv=5, #Determines the cross-validation splitting strategy.
  #  n_jobs=4 #number of jobs to run parallel
#)

#didn't get the answer, how to handle the search_space in this case (if possible at all)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('model', SVR()) 
])

lin_search = {
    'model' : Categorical([LinearRegression()]),
}

linear =  BayesSearchCV(
    pipe,
    [(lin_search,1)], 
    cv=5,
    n_iter=32,
    random_state=0,
    n_jobs=4, #parallelize
    scoring='neg_mean_squared_error'
)

In [ ]:
linear.fit(X_train, y_train)

In [ ]:
linear.best_score_